In [280]:
import pandas as pd
import sqlite3
from Levenshtein import distance
from fuzzywuzzy import fuzz

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
df = pd.read_sql('SELECT * FROM words', conn)

# close the database connection
conn.close()

In [259]:
df.columns

Index(['word', 'type', 'translation', 'example', 'example_translation',
       'russian', 'appear', 'trial_d', 'trial_r', 'success', 'weight',
       'word_index', 'difficulty', 'wd'],
      dtype='object')

In [261]:
df = df.drop(columns=['difficulty', 'wd'])

In [262]:
columns = list(df.columns)

df['Length_word'] = df['word'].str.len()
df['translation'] = df['translation'].str.split(',').str[0]
df['Length_translation'] = df['translation'].str.len()
# Calculate Levenshtein Distance between word and translation
df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)
    
# dividing into quantiles
df['d1'] = pd.qcut(df['Length_word'], q=4, labels=[0, 1, 2, 3])
df['d1'] = df['d1'].astype(int)
df['d2'] = pd.qcut(df['Length_translation'], q=3, labels=[0, 1, 2])
df['d2'] = df['d2'].astype(int)
df['d3'] = pd.qcut(df['Similarity'], q=5, labels=[4, 3, 2, 1, 0])
df['d3'] = df['d3'].astype(int)
df['d4'] = pd.cut(df['weight'], bins=3, labels=[1, 2, 3])
df['d4'] = df['d4'].astype(int)

df['difficulty'] = df['d1'] + df['d2'] + df['d3']
df['wd'] = df['d1'] + df['d2'] + df['d3'] + df['d4']

columns = columns + ['difficulty', 'wd']

df = df.loc[:, columns]

In [263]:
df['wd'].unique()

array([ 3,  1,  7,  4,  6,  5,  2,  8,  9, 10, 11, 12])

In [264]:
def xlstosql(df):
    df_name = df.name
    # connect to the SQLite database
    conn = sqlite3.connect(f'data_files/{df_name}.db')
    # insert the data from the dataframe into the database table
    df.to_sql(f'{df_name}', conn, if_exists='replace', index=False)
    # close the database connection
    conn.close()

In [265]:
df.name = 'words'
xlstosql(df)

In [213]:
dif_df = df.loc[:, ['word_index', 'difficulty']]

In [ ]:
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

cursor.execute("ALTER TABLE words ADD COLUMN word_index INTEGER")

cursor.execute("SELECT rowid FROM words")
rows = cursor.fetchall()

for i, row in enumerate(rows, start=1):
    cursor.execute("UPDATE words SET word_index = ? WHERE rowid = ?", (i, row[0]))
conn.commit()

# Сохранение изменений и закрытие соединения
conn.close()

In [237]:
def difficulty(df, level='standard'):
    
    columns = list(df.columns)
    
    df['Length_word'] = df['word'].str.len()
    df['translation'] = df['translation'].str.split(',').str[0]
    df['Length_translation'] = df['translation'].str.len()
    # Calculate Levenshtein Distance between word and translation
    df['Similarity'] = df.apply(lambda row: fuzz.ratio(row['word'], row['translation']), axis=1)
    
    # dividing into quantiles
    df['d1'] = pd.qcut(df['Length_word'], q=4, labels=[0, 1, 2, 3])
    df['d1'] = df['d1'].astype(int)
    df['d2'] = pd.qcut(df['Length_translation'], q=3, labels=[0, 1, 2])
    df['d2'] = df['d2'].astype(int)
    df['d3'] = pd.qcut(df['Similarity'], q=5, labels=[4, 3, 2, 1, 0])
    df['d3'] = df['d3'].astype(int)
    df['d4'] = pd.qcut(df['weight'], bins=3, labels=[1, 2, 3], duplicates='drop')
    df['d4'] = df['d4'].astype(int)
    
    df['difficulty'] = df['d1'] + df['d2'] + df['d3'] + df['d4']
    
    
    
    df = df.loc[:, columns]
    return df


In [198]:
# Calculate word lengths and divide into three bins
#df['Difficulty'] = pd.cut(df['Length'], bins=3, labels=['Easy', 'Medium', 'Hard'])

In [228]:
test_list = []

for i in range(0,101):
    test_list.append(i)

In [229]:
tt=pd.DataFrame()
tt['count'] = test_list


In [230]:
tt['dif'] = pd.cut(test_list, bins=3, labels=['Easy', 'Medium', 'Hard'])

In [266]:
writer = pd.ExcelWriter('data_files/tdddt.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [138]:
conn2 = sqlite3.connect('data_files/lessons.db')
lesson = pd.read_sql('SELECT * FROM lessons', conn2)
conn2.close()

In [139]:
lesson_df = lesson[lesson['known'] != 25]
lesson_df = lesson_df.loc[:, ['r', 'list_of_words', 'points']]

In [140]:
# Create a sample difficulty DataFrame
difficulty_df = df.loc[:,['word', 'difficulty']]

# Split 'list of words' column and retrieve difficulties
lesson_df['words'] = lesson_df['list_of_words'].str.split(';')
lesson_df['words'] = lesson_df['words'].apply(lambda x: [word.strip() for word in x])
lesson_df['dif'] = lesson_df['words'].apply(lambda x: sum(difficulty_df[difficulty_df['word'].isin(x)]['difficulty']))

In [141]:
# Remove the intermediate 'words' column
lesson_df.drop('words', axis=1, inplace=True)

In [142]:
writer = pd.ExcelWriter('data_files/test_check.xlsx')
lesson_df.to_excel(writer, sheet_name='update')
writer.close()

In [143]:
# Add a unique key column based on index
df['key'] = df.index

# Define the desired distribution of words by difficulty
difficulty_distribution_easy = {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6:2, 7:1, 8:1 , 9:1, 10:0}
difficulty_distribution = {1: 2, 2: 2, 3: 3, 4: 3, 5: 3, 6: 3, 7:3, 8:2 ,9:2, 10:2 }
difficulty_distribution_hard = {1: 0, 2: 1, 3: 1, 4: 1, 5: 2, 6: 4, 7:4, 8:4 ,9:4, 10:4 }
difficulty_distribution_hardest = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 5, 7:5, 8:5 ,9:5, 10:5 }

# Sample words from each difficulty level based on difficulty and weight
selected_words = []
for difficulty, count in difficulty_distribution.items():
    words_subset = df[(df['wb'] == difficulty)]
    words_subset = words_subset.sample(n=count, weights=words_subset['weight'])
    selected_words.append(words_subset)
    
# Concatenate the selected words into a new DataFrame
new_words_df = pd.concat(selected_words)

writer = pd.ExcelWriter('data_files/new.xlsx')
new_words_df.to_excel(writer, sheet_name='update')
writer.close()

In [122]:
# Reset the index of the new DataFrame
new_words_df.reset_index(drop=True, inplace=True)

# Update the parameters of the selected words
new_words_df['weight'] = new_words_df['weight'] * 2  # Update the weight parameter as an example

# Merge the updated words with the original DataFrame based on the key column
df = pd.merge(df, new_words_df, on='key', how='left')

# Fill missing values in the original DataFrame with the updated values
df['difficulty_y'].fillna(df['difficulty_x'], inplace=True)
df['weight_y'].fillna(df['weight_x'], inplace=True)

# Drop redundant columns from the merged DataFrame
df.drop(['difficulty_x', 'weight_x', 'key'], axis=1, inplace=True)
df.rename(columns={'difficulty_y': 'difficulty', 'weight_y': 'weight'}, inplace=True)

writer = pd.ExcelWriter('data_files/final.xlsx')
df.to_excel(writer, sheet_name='update')
writer.close()

In [274]:
', '.join(list(df.columns))

'word, type, translation, example, example_translation, russian, appear, trial_d, trial_r, success, weight, word_index, difficulty, wd'

In [305]:
import sqlite3
import random

difficulty_distribution = {1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 2, 11: 1, 12: 1}

def sql_text(dffty, limit):
    text = f"""
    SELECT word, type, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight, word_index, difficulty, wd
    FROM words
    WHERE wd = {dffty}
    ORDER BY weight DESC, RANDOM()
    LIMIT {limit};
    """
    return text

def loadData(source, final='no'):
    # connect to the SQLite database and read the data into a pandas dataframe
    conn = sqlite3.connect('data_files/words.db')
    
    
    if source == 'word' and final == 'no':
        cursor = conn.cursor()
        # Sample words from each difficulty level based on difficulty and weight
        selected_words = []
        for difficulty, count in difficulty_distribution.items():
            # Retrieve 5 easy words
            words_query = sql_text(difficulty, count)
            cursor.execute(words_query)
            selected_words = selected_words + cursor.fetchall()
        return selected_words
    else:
        df = pd.read_sql(f'SELECT * FROM {source}s', conn)
        df = df.loc[:, f'{source}':]
        
    # close the database connection
    conn.close()
    

combined_data = loadData('word')

    
class Words(object):

    """The class for all words from dictionary. Each word a type, a translation, an example,
    a translation of an example, a translation to Russian, also additional parameters which would be created after
    the first run (quantity of appearances in lessons, trials of direct and indirect translation,
    success attempts, also weight, more weight - more often words appear"""

    def __init__(self, word, typ, translation, russian, example, example_translation, appear, trial_d, trial_r, success,
                 weight, word_index, difficulty, wd):
        self.word = word
        self.typ = typ
        self.translation = translation
        self.example = example
        self.example_translation = example_translation
        self.russian = russian
        self.appear = appear
        self.trial_d = trial_d
        self.trial_r = trial_r
        self.success = success
        self.weight = weight
        self.word_index = word_index
        self.difficulty = difficulty
        self.wd = wd


# Create Word objects from the combined data
word_objects = []
for row in combined_data:
    word = Words(*row)
    word_objects.append(word)

# Print the created Word objects
for word in word_objects:
    print(f"Word: {word.word}, Translation: {word.translation}, Weight: {word.weight}, Difficulty: {word.wd}")


Word: skiën, Translation: skiing, Weight: 33.33, Difficulty: 1
Word: robot, Translation: robot, Weight: 50.0, Difficulty: 2
Word: mijl, Translation: mile, Weight: 100.0, Difficulty: 3
Word: minuut, Translation: minute, Weight: 100.0, Difficulty: 3
Word: methode, Translation: method, Weight: 100.0, Difficulty: 4
Word: liegen, Translation: to lie, Weight: 100.0, Difficulty: 4
Word: goederen, Translation: goods, Weight: 100.0, Difficulty: 5
Word: printer, Translation: printer, Weight: 100.0, Difficulty: 5
Word: verstand, Translation: head, Weight: 100.0, Difficulty: 5
Word: daar, Translation: there, Weight: 100.0, Difficulty: 6
Word: aanvoerder, Translation: leader, Weight: 100.0, Difficulty: 6
Word: identiteit, Translation: identity, Weight: 100.0, Difficulty: 6
Word: lijk, Translation: corpse, Weight: 100.0, Difficulty: 7
Word: heelal, Translation: universe, Weight: 100.0, Difficulty: 7
Word: minderjarig, Translation: underage, Weight: 100.0, Difficulty: 7
Word: slijter, Translation: li

In [291]:
def sql_text(dffty, limit):
    text = f"""
    SELECT word, type, translation, russian, example, example_translation, appear, trial_d, trial_r, success, weight, word_index, difficulty, wd
    FROM words
    WHERE wd = {dffty}
    ORDER BY weight DESC, RANDOM()
    LIMIT {limit};
    """
    return text


# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

# Retrieve 5 easy words
easy_words_query = sql_text(2, 15)
cursor.execute(easy_words_query)
easy_words = cursor.fetchall()

# Retrieve 5 standard words
standard_words_query = sql_text(6, 20)
cursor.execute(standard_words_query)
standard_words = cursor.fetchall()

# Retrieve 5 hard words
hard_words_query = sql_text(10, 5)
cursor.execute(hard_words_query)
hard_words = cursor.fetchall()

# Close the database connection
conn.close()

# Print the retrieved words

print("\nEasy Words:")
for row in easy_words:
    print(row)

print("\nStandard Words:")
for row in standard_words:
    print(row)

print("\nHard Words:")
for row in hard_words:
    print(row)


Easy Words:
('robot', 'noun', 'robot', None, None, 'робот', 1, 2, 2, 2, 50.0, 7905, 0, 2)
('dubbel', 'adj', 'double', 'De dubbele plaatjes ruilde hij op school met andere verzamelaars.', 'He traded the duplicate plates at school with other collectors.', 'двойной', 2, 3, 5, 4, 50.0, 800, 0, 2)
('hoer', 'noun, de(f)', 'whore', 'Dit was de buurt van dronkenlappen, matrozen en hoeren.', 'This was the neighborhood of drunks, sailors and whores.', 'шлюха', 2, 2, 5, 4, 42.86, 1512, 0, 2)
('moslim', 'noun, de(m)', 'Muslim', 'Christenen, moslims, joden en boeddhisten deden afgelopen zaterdag mee aan de bijeenkomst.', 'Christians, Muslims, Jews and Buddhists participated in the meeting last Saturday.', 'мусульманин', 2, 3, 4, 4, 42.86, 2432, 0, 2)
('elleboog', 'noun, de(m)', 'elbow', 'Hij plantte zijn elleboog op zijn dij en liet zijn hoofd op zijn gebalde vuist rusten.', 'He planted his elbow on his thigh and rested his head on his clenched fist.', 'локоть', 3, 3, 6, 6, 33.33, 1410, 1, 2)
('ra

In [292]:

text = """
    SELECT weight, word_index, wd
    FROM words
    """

# connect to the SQLite database and read the data into a pandas dataframe
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

# Retrieve 5 easy words
easy_words_query = text
cursor.execute(easy_words_query)
easy_words = cursor.fetchall()

# Close the database connection
conn.close()

In [295]:
easy_words[1]

(0.0, 2, 3)

In [306]:
conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

v1 = 100
v2 = 100
v3 = 100
v4 = 100
v5 = 100

cursor.execute("UPDATE words SET appear = ?, trial_d = ?, trial_r = ?, success = ?, wd = ?  WHERE word_index = ?",
            (v1, v2, v3, v4, v5, 1))

    # Commit the changes and close the connection
conn.commit()
conn.close()

In [311]:
text = """

UPDATE words
SET weight = 
             CASE
                WHEN trial_d = 0 AND trial_r = 0 THEN ROUND(100, 2)
                ELSE ROUND((100 - (success / (trial_d + trial_r)) * 100), 2)
             END;
"""

conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

cursor.execute(text)
conn.commit()
conn.close()

In [329]:
text = """

UPDATE words
SET weight = 
    CASE
        WHEN trial_d = 0 AND trial_r = 0 THEN ROUND(100, 2)
        ELSE ROUND((100 - success * 100 /(trial_d + trial_r)), 2)
    END;
"""

conn = sqlite3.connect('data_files/words.db')
cursor = conn.cursor()

cursor.execute(text)
conn.commit()
conn.close()

In [332]:
conn = sqlite3.connect('data_files/lessons.db')
cursor = conn.cursor()

cursor.execute("SELECT lesson FROM lessons WHERE lesson IS NOT NULL ORDER BY lesson DESC LIMIT 1")
last_lesson = cursor.fetchone()[0]
l1 = last_lesson + 1
l2 = 0
l3 = 0
l4 = 0
l5 = 0
l6 = 0
l7 = 0
l8 = 0
l9 = 0
l10 = 0
    
cols = 'lesson, start, inter, finish, known, points, length, time, list_of_words, r'

cursor.execute(f"INSERT INTO lessons ({cols}) VALUES (?,?,?,?,?,?,?,?,?,?)", (l1,l2,l3,l4,l5,l6,l7,l8,l9,l10))

conn.commit()
conn.close()